In [4]:
!pip install pandas numpy matplotlib seaborn scikit-learn torch torchvision transformers datasets tqdm collections

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from collections import defaultdict

In [11]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [12]:
# Load and preprocess
df = pd.read_csv('final_training_data.csv')
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

# Normalize labels to 0-indexed
label_col = 'label_num'
label_mapping = {old: i for i, old in enumerate(sorted(df[label_col].unique()))}
df[label_col] = df[label_col].map(label_mapping)
all_metrics = defaultdict(list)
classwise_metrics = defaultdict(lambda: defaultdict(list))  # {class_id: {prec: [], rec: [], f1: []}}

In [13]:
class CNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=3, padding=1),  # Change here
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),

            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [14]:
# Training loop per station
for lat, lon in tqdm(stations, desc="Stations"):
    station_df = df[(df['station_lat'] == lat) & (df['station_lon'] == lon)]
    if station_df[label_col].nunique() < 2:
        continue  # Skip if only one class present

    # Features and labels
    X = station_df.drop(columns=[label_col, 'station_lat', 'station_lon']).values
    y = station_df[label_col].values

    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

        # Train/test split
    split_idx = int(len(X) * 0.8)  # first 80% for training, last 20% for testing
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(2)  # (samples, features, 1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(2)    # same for test

    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=False)

    model = CNN(input_channels=X.shape[1], num_classes=len(np.unique(y))).to('cpu')

    unique_classes = np.unique(y_train)
    if len(unique_classes) < 2:
        continue  # Skip stations with only one class

    # Compute class weights for current station
    class_weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=y_train)
    # Create full weight vector of size [num_classes] with zeros
    full_weights = torch.zeros(len(np.unique(y)))  # Use global num_classes from earlier
    # Fill weights for present classes
    for i, cls in enumerate(unique_classes):
        full_weights[cls] = class_weights[i]

    criterion = nn.CrossEntropyLoss(weight=full_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    # Training
    for epoch in range(20):
        model.train()
        epoch_loss = 0.0
        for xb, yb in train_loader:
            outputs = model(xb)

            # NaN check
            if torch.isnan(outputs).any():
                print(f"NaN in outputs at station ({lat:.4f}, {lon:.4f})")
                continue

            loss = criterion(outputs, yb)
            if torch.isnan(loss):
                print(f"NaN in loss at station ({lat:.4f}, {lon:.4f})")
                continue

            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item()

        print(f"[{lat:.4f}, {lon:.4f}] Epoch {epoch+1}/5, Loss: {epoch_loss:.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor).argmax(dim=1).cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()

        # Overall metrics
        acc = accuracy_score(y_true, preds)
        prec = precision_score(y_true, preds, average='macro', zero_division=0)
        rec = recall_score(y_true, preds, average='macro', zero_division=0)
        f1 = f1_score(y_true, preds, average='macro', zero_division=0)

        print(f"📍 Station ({lat:.4f}, {lon:.4f})")
        print(f"  Accuracy : {acc:.4f}")
        print(f"  Precision: {prec:.4f}")
        print(f"  Recall   : {rec:.4f}")
        print(f"  F1-score : {f1:.4f}")

        for cls in class_labels:
            cls_mask = (y_true == cls)
            cls_preds = (preds == cls)
            tp = np.sum(cls_mask & cls_preds)
            fp = np.sum(~cls_mask & cls_preds)
            fn = np.sum(cls_mask & ~cls_preds)

            cls_prec = tp / (tp + fp + 1e-8)
            cls_rec = tp / (tp + fn + 1e-8)
            cls_f1 = 2 * cls_prec * cls_rec / (cls_prec + cls_rec + 1e-8)

            print(f"    Class {cls}: Precision={cls_prec:.4f}, Recall={cls_rec:.4f}, F1-score={cls_f1:.4f}")

            # Log classwise metrics
            classwise_metrics[cls]['precision'].append(cls_prec)
            classwise_metrics[cls]['recall'].append(cls_rec)
            classwise_metrics[cls]['f1'].append(cls_f1)


        # Log metrics for averaging
        all_metrics['accuracy'].append(acc)
        all_metrics['precision'].append(prec)
        all_metrics['recall'].append(rec)
        all_metrics['f1'].append(f1)


Stations:   0%|          | 0/217 [00:00<?, ?it/s]

[19.0500, 74.4800] Epoch 1/5, Loss: 7.6838
[19.0500, 74.4800] Epoch 2/5, Loss: 7.3042
[19.0500, 74.4800] Epoch 3/5, Loss: 7.1317
[19.0500, 74.4800] Epoch 4/5, Loss: 7.0319
[19.0500, 74.4800] Epoch 5/5, Loss: 6.8700
[19.0500, 74.4800] Epoch 6/5, Loss: 6.6910
[19.0500, 74.4800] Epoch 7/5, Loss: 6.5623
[19.0500, 74.4800] Epoch 8/5, Loss: 6.4645
[19.0500, 74.4800] Epoch 9/5, Loss: 6.0721
[19.0500, 74.4800] Epoch 10/5, Loss: 6.0548
[19.0500, 74.4800] Epoch 11/5, Loss: 5.8641
[19.0500, 74.4800] Epoch 12/5, Loss: 5.8069
[19.0500, 74.4800] Epoch 13/5, Loss: 5.9074
[19.0500, 74.4800] Epoch 14/5, Loss: 5.3325
[19.0500, 74.4800] Epoch 15/5, Loss: 5.2674
[19.0500, 74.4800] Epoch 16/5, Loss: 5.1516
[19.0500, 74.4800] Epoch 17/5, Loss: 4.6168
[19.0500, 74.4800] Epoch 18/5, Loss: 4.3549


Stations:   0%|          | 1/217 [00:01<03:41,  1.03s/it]

[19.0500, 74.4800] Epoch 19/5, Loss: 4.5424
[19.0500, 74.4800] Epoch 20/5, Loss: 4.2020
📍 Station (19.0500, 74.4800)
  Accuracy : 0.9425
  Precision: 0.7265
  Recall   : 0.5939
  F1-score : 0.6279
    Class 0: Precision=0.9529, Recall=0.9878, F1-score=0.9701
    Class 1: Precision=0.5000, Recall=0.2000, F1-score=0.2857
[18.4200, 76.5600] Epoch 1/5, Loss: 12.6798
[18.4200, 76.5600] Epoch 2/5, Loss: 12.1891
[18.4200, 76.5600] Epoch 3/5, Loss: 11.3007
[18.4200, 76.5600] Epoch 4/5, Loss: 10.9996
[18.4200, 76.5600] Epoch 5/5, Loss: 10.7824
[18.4200, 76.5600] Epoch 6/5, Loss: 10.5994
[18.4200, 76.5600] Epoch 7/5, Loss: 10.0283
[18.4200, 76.5600] Epoch 8/5, Loss: 9.7062
[18.4200, 76.5600] Epoch 9/5, Loss: 9.5833
[18.4200, 76.5600] Epoch 10/5, Loss: 9.2844
[18.4200, 76.5600] Epoch 11/5, Loss: 8.7509
[18.4200, 76.5600] Epoch 12/5, Loss: 8.4295
[18.4200, 76.5600] Epoch 13/5, Loss: 7.8706
[18.4200, 76.5600] Epoch 14/5, Loss: 7.4260
[18.4200, 76.5600] Epoch 15/5, Loss: 7.5919
[18.4200, 76.5600] Ep

Stations:   1%|          | 2/217 [00:02<04:42,  1.31s/it]

...

Stations:  56%|█████▌    | 121/217 [02:42<02:12,  1.38s/it]

[18.1900, 72.5800] Epoch 17/5, Loss: 8.1547
[18.1900, 72.5800] Epoch 18/5, Loss: 7.8342
[18.1900, 72.5800] Epoch 19/5, Loss: 7.7360
[18.1900, 72.5800] Epoch 20/5, Loss: 7.6392
📍 Station (18.1900, 72.5800)
  Accuracy : 0.6738
  Precision: 0.6617
  Recall   : 0.7160
  F1-score : 0.6467
    Class 0: Precision=0.9054, Recall=0.6321, F1-score=0.7444
    Class 1: Precision=0.4179, Recall=0.8000, F1-score=0.5490
[18.5000, 77.3100] Epoch 1/5, Loss: 12.8228
[18.5000, 77.3100] Epoch 2/5, Loss: 11.9606
[18.5000, 77.3100] Epoch 3/5, Loss: 11.7390
[18.5000, 77.3100] Epoch 4/5, Loss: 11.5271
[18.5000, 77.3100] Epoch 5/5, Loss: 10.7699
[18.5000, 77.3100] Epoch 6/5, Loss: 10.7376
[18.5000, 77.3100] Epoch 7/5, Loss: 10.3602
[18.5000, 77.3100] Epoch 8/5, Loss: 9.8848
[18.5000, 77.3100] Epoch 9/5, Loss: 9.5593
[18.5000, 77.3100] Epoch 10/5, Loss: 9.1974
[18.5000, 77.3100] Epoch 11/5, Loss: 8.9282
[18.5000, 77.3100] Epoch 12/5, Loss: 8.3782
[18.5000, 77.3100] Epoch 13/5, Loss: 8.3283
[18.5000, 77.3100] Ep

Stations:  56%|█████▌    | 122/217 [02:43<02:07,  1.34s/it]

[18.5000, 77.3100] Epoch 20/5, Loss: 5.7052
📍 Station (18.5000, 77.3100)
  Accuracy : 0.8777
  Precision: 0.6466
  Recall   : 0.6566
  F1-score : 0.6513
    Class 0: Precision=0.9360, Recall=0.9286, F1-score=0.9323
    Class 1: Precision=0.3571, Recall=0.3846, F1-score=0.3704


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 32, 1])

In [15]:
# After the for-loop over all stations:
print("\n🔚 Average Metrics Across All Stations:")
print(f"  Accuracy : {np.mean(all_metrics['accuracy']):.4f}")
print(f"  Precision: {np.mean(all_metrics['precision']):.4f}")
print(f"  Recall   : {np.mean(all_metrics['recall']):.4f}")
print(f"  F1-score : {np.mean(all_metrics['f1']):.4f}")

print("\n📊 Class-wise Average Metrics Across All Stations:")
for cls in sorted(classwise_metrics.keys()):
    prec = np.mean(classwise_metrics[cls]['precision'])
    rec = np.mean(classwise_metrics[cls]['recall'])
    f1 = np.mean(classwise_metrics[cls]['f1'])
    print(f"  Class {cls}: Precision={prec:.4f}, Recall={rec:.4f}, F1-score={f1:.4f}")



🔚 Average Metrics Across All Stations:
  Accuracy : 0.8351
  Precision: 0.5993
  Recall   : 0.6111
  F1-score : 0.5920

📊 Class-wise Average Metrics Across All Stations:
  Class 0: Precision=0.9112, Recall=0.8647, F1-score=0.8831
  Class 1: Precision=0.2709, Recall=0.3411, F1-score=0.2845
